# Exercise 4

## Helper functions

In [1]:
import numpy as np
import cv2 
import os
import scipy
import matplotlib.pyplot as plt
import matplotlib

def load_im(path : str) -> np.ndarray:
    
    im = cv2.imread(path)[:, :, ::-1]
    im = im.astype(np.float64) / 255
    
    return im


def pi(points : np.ndarray) -> np.ndarray:
    """
        Converts from homogeneous to inhomogeneous coordinates
    """
    p = points[:-1]/points[-1]
    
    return p


def piInv(points : np.ndarray) -> np.ndarray:
    """
        Converts from inhomogeneous to homogeneous coordinates
    """
    
    # Gets the amount of points by using shape
    _, num_points = points.shape
    
    # Stacks the scale s at the bottom of the matrix
    ph = np.vstack((points, np.ones(num_points)))
    
    return ph


def projectPoints(K, Rt, Q):
    
    Q_hom = piInv(Q)
    points = K @ Rt @ Q_hom
    points_inhom = pi(points)
    
    return points_inhom


def hest(q1, q2) -> np.ndarray:
    """
        Takes two points in 2D and returns the estimated homography matrix.
    """
    
    if len(q1) != len(q2):
        raise ValueError("There must be an equal amount of points in the two sets!")
    
    Bi = []
    for i in range(q1.shape[1]):
        qi = q1[:,i]   # <-- getting the first column
        
        # Creating that weird qx matrix for the Kronecker product
        q1x = np.array(
            [[0,        -1, qi[1]],
             [1,        0, -qi[0]],
             [-qi[1], qi[0], 0]]
        )
        
        q2t_hom = q2[:, i].reshape(-1, 1) # <-- getting the first column and reshaping does for dim: (1, ) -> (1,1)
        Bi.append(np.kron(q2t_hom.T, q1x)) # <-- formula follows that of week 2, slide 56
        # print(np.kron(q2t_hom.T, q1x).shape)
       
    B = np.concatenate(Bi, axis=0)
    
    # Some TA prooved that it was unneseccary to find their dot product
    #BtB = B.T @ B
    V, Lambda, Vt = np.linalg.svd(B)
    Ht = Vt[-1, :]
    
    Ht = np.reshape(Ht, (3, 3))
    H = Ht.T
    
    return H
    
    
def crossOp(p : np.ndarray) -> np.ndarray:
    """
        One of Them weird functions. It takes in a 3D vector and then returns
        some gnarly matrix.
    """
    p = p.flatten()
    if p.size != 3:
        raise Exception("Invalid input, vector must be exactly 3D.")
    
    x, y, z = p
    px = np.array(
        [[0, -z, y],
         [z, 0, -x],
         [-y, x, 0]]
    )
    
    return px


def computeFundamentalMatrix(K1 : np.ndarray, K2 : np.ndarray, R2 : np.ndarray, t2 : np.ndarray) -> np.ndarray:
    """
        Computing the fundamental matrix between two camera matrices K1 & K2.
    """
    t2x = CrossOp(t2)

    E = t2x @ R2

    K1inv = np.linalg.inv(K1)
    K2inv = np.linalg.inv(K2)

    F = K1inv.T @ E @ K2inv
    
    return F


def fancyRotate(theta_x, theta_y, theta_z):
    """
        Does the rotation matrix that we have seen a few times.
        E.g. Exercises week 4, eq(12).
    """
    from scipy.spatial.transform import Rotation
    
    R = Rotation.from_euler("xyz", [theta_x, theta_y, theta_z]).as_matrix()
    
    return R




## Initial setup
Its propaply good to remember that:
$$
    \begin{align*}
        \pmb{p}_h &= \pmb{K} \pmb{P}_{cam}
        \\
        &=  \pmb{K} \left[ \pmb{R} \pmb{t} \right] \pmb{P}_h
    \end{align*}
$$

Where:
$$
    \begin{equation*}
        \mathcal{P} = \pmb{K} \left[ \pmb{R} \pmb{t} \right]
    \end{equation*}
$$

## Ex. 4.1
Fin the projection matrix $\mathcal{P}$ and the projections $\pmb{q}$.

In this section consider the 3D points:
$$
    \begin{equation*}
        \pmb{Q}_{ijk} = 
        \begin{bmatrix}
            i   \\
            j   \\
            k
        \end{bmatrix}
        \qquad \text{for } i \in \{0, 1\}, \quad j \in \{0, 1\}, \quad k \in \{0, 1\}.
    \end{equation*}
$$

We also have some other things which are defined in the code

In [2]:
f = 1000
resolution = (1920, 1080)

R = np.array(
    [[np.sqrt(1/2), -np.sqrt(1/2), 0],
     [np.sqrt(1/2),  np.sqrt(1/2), 0],
     [0,                        0, 1]]
)

t = np.array([0, 0, 10]).reshape(-1, 1)

Q = np.zeros((3, 8))

count = 0

for i in range(2):
    for j in range(2):
        for k in range(2):
            Q[:, count] = np.array([i, j, k])
            count += 1

print(Q)

[[0. 0. 0. 0. 1. 1. 1. 1.]
 [0. 0. 1. 1. 0. 0. 1. 1.]
 [0. 1. 0. 1. 0. 1. 0. 1.]]


### Response

In [3]:
Rt = np.concatenate((R, t), axis=1)

# We define K as per slide 12 of week 2
alpha = 1
beta = 0

deltax = resolution[0] / 2
deltay = resolution[1] / 2

K = np.array(
    [[f, beta*f, deltax],
     [0, alpha*f, deltay],
     [0, 0, 1]]
    )

P = K @ Rt

qs = projectPoints(K, Rt, Q)

print(f"Yields a kamera matrix of: \n{K}")
print(f"\nand a projection matrix of: \n{P}")
print(f"\nfinally... the qs: \n{qs.T}")

Yields a kamera matrix of: 
[[1000.    0.  960.]
 [   0. 1000.  540.]
 [   0.    0.    1.]]

and a projection matrix of: 
[[ 7.07106781e+02 -7.07106781e+02  9.60000000e+02  9.60000000e+03]
 [ 7.07106781e+02  7.07106781e+02  5.40000000e+02  5.40000000e+03]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  1.00000000e+01]]

finally... the qs: 
[[ 960.          540.        ]
 [ 960.          540.        ]
 [ 889.28932188  610.71067812]
 [ 895.71756535  604.28243465]
 [1030.71067812  610.71067812]
 [1024.28243465  604.28243465]
 [ 960.          681.42135624]
 [ 960.          668.56486931]]


## Ex. 4.2

Write a function `pest` that uses $\pmb{Q}$ and $\pmb{q}$ to estimate $\mathcal{P}$ with the DLT.

Also find the RMSE.

(and more...)

### Response

In [4]:
def pest(Q : np.ndarray, q : np.ndarray) -> np.ndarray:
    """
        Takes in:
            A matrix of 3D points: Q
            
            A homogenous point: q
        
        Returns:
            A projection matrix: P   
    """
    
    Bi = []
    for i in range(q.shape[1]):
        qi = q[:, i].reshape(-1, 1)
        Qi = Q[:, i].reshape(-1, 1)
        
        q_hom = piInv(qi)
        Q_hom = piInv(Qi)
        
        qx = crossOp(q_hom)
        
        Bi.append(np.kron(Q_hom, qx).T) # <-- formula follows that of week 4, slide 6
       
    B = np.concatenate(Bi, axis=0)
    #print(B.shape)
    # Some TA prooved that it was unneseccary to find their dot product
    #BtB = B.T @ B
    _, _, Qt = np.linalg.svd(B)
    Qt = Qt[-1, :]
    
    Qt = np.reshape(Qt, (4, 3))
    Q = Qt.T
    
    return Q


def RMSE(q : np.ndarray, q_tilde : np.ndarray) -> np.ndarray:
    m = q.shape[1]
    return np.sqrt(np.sum(np.power(q_tilde - q, 2)) / m)
    
    

Is it the same P?

In [5]:
P_est = pest(Q, qs)

p_est_scaled = P_est * (P[0, 0]/P_est[0, 0])

print("This matrix should be close to zero (at all indices), since it is the true subtracted from the approximated")
print(p_est_scaled - P)

This matrix should be close to zero (at all indices), since it is the true subtracted from the approximated
[[ 0.00000000e+00  2.89901436e-11  3.02406988e-11 -1.09139364e-11]
 [-1.13686838e-12  1.85309545e-11  2.06910045e-11 -8.18545232e-12]
 [ 1.32870791e-12 -3.50752874e-12  2.46580534e-12 -1.32693856e-12]]


In [6]:
P_est = pest(Q, qs)
q_est = P_est @ piInv(Q)

error = RMSE(qs, pi(q_est)) # <-- apparently we need to make q_est inhomogenous as they could otherwise have scaling that is way off



## Ex. 4.3 
Make a `checkerboardPoints(n, m)` function that returns the 3D points:

$$
    \begin{equation*}
        \pmb{Q}_{ij} = 
        \begin{bmatrix}
            i - \frac{n - 1}{2}  \\
            j - \frac{m - 1}{2}  \\
            0
        \end{bmatrix}
        \qquad
        \text{for } i \in \{0, ..., n-1\}, \quad j \in \{0, ..., m - 1\}
    \end{equation*}
$$

### Response

In [7]:
def checkerboardPoints(n : int, m : int) -> np.ndarray:
    """
        Takes:
            an integer: n
            
            an integer: m
        
        Returns:
            Weird matrix of size 3 x (n * m) : Q
            (As defined per week 4 exercises, eq (7))
        
        Idiot code* explained:
            We can't numpy.hstack to an empty array, so we initialize the first column, then
            when we return, we just return all of the matrix except for the first column.
    """
    
    Q = np.zeros((3, 1))    # <-- idiot code (1/2)
    
    for i in range(n):
        for j in range(m):
            temp = np.array([i - (n - 1)/2, j - (m - 1)/2, 0]).reshape(-1, 1)
            Q = np.hstack((Q, temp))
    
    Q = Q[:, 1:]    # <-- iditot code (2/2)
    
    return Q

## Ex. 4.4
A lot of funky stuff

### Response

In [8]:
%matplotlib qt
N = 10
M = 20
Q_omega = checkerboardPoints(N, M)

Qa = fancyRotate(np.pi/ 10, 0, 0) @ Q_omega
Qb = fancyRotate(0, 0, 0) @ Q_omega
Qc = fancyRotate(- np.pi/ 10, 0, 0) @ Q_omega


fig = plt.figure(figsize = (16, 9))
ax = fig.add_subplot(projection='3d')   # <-- this forces the figure to be 3D

ax.scatter(Qa[0, :], Qa[1, :], Qa[2, :])
ax.scatter(Qb[0, :], Qb[1, :], Qb[2, :])
ax.scatter(Qc[0, :], Qc[1, :], Qc[2, :])

plt.show()


Now we project to image plane!

In [9]:
projectInhom2Inhom = lambda Q : pi(P @ piInv(Q))
qa = projectInhom2Inhom(Qa)
qb = projectInhom2Inhom(Qb)
qc = projectInhom2Inhom(Qc)


plt.figure(figsize=(16, 9))

plt.scatter(qa[0, :], qa[1, :])
plt.scatter(qb[0, :], qb[1, :])
plt.scatter(qc[0, :], qc[1, :])

plt.xlim(0, 1920)
plt.ylim(0, 1080)

plt.show()

## Ex. 4.5
Define a function `estimateHomographies(Q_omega, qs)`.

...Easier said than done :/

### Response

In [10]:
def estimateHomographies(Q_omega : np.ndarray, qs : list) -> list:
    """
        Takes:
            A Q matrix: Q_omega
            
            A list of qs for which we find their homographies: qs
            
        Returns:
            A list of homographies corresponding to qs: Hs
    """
    
    Hs = []
    for q in qs:
        H = hest(q, Q_omega[[0,1,3], :]) # <-- Only X, Y coordinates of Q_omega (and the scale at the fourth coordinate)
        Hs.append(H)
    
    return Hs
    


Testing

In [11]:
### `hest` skal måske skrives om.... ###
qs = [piInv(qa), piInv(qb), piInv(qc)]
Q_omega_hom = piInv(Q_omega)
Hs = estimateHomographies(Q_omega_hom, qs)

for i in range(len(qs)):
    q_est = Hs[i] @ Q_omega_hom[[0,1,3], :] # <-- Only X, Y coordinates of Q_omega (and the scale at the fourth coordinate)
    error = RMSE(pi(qs[i]), pi(q_est)) # <-- apparently we need to make q_est & qs inhomogenous as they could otherwise have scaling that is way off
    print(f"Error{i + 1}: {error}")


Error1: 1.2379686963917365e-10
Error2: 4.697565415745792e-10
Error3: 5.754668572505472e-10


Got the same results as Jonas:

In [12]:
for hom in Hs:
    Hi_scaled = hom / hom[0, 0]
    print(Hi_scaled)
    print()

[[ 1.00000000e+00 -5.31521133e-01  1.35764502e+01]
 [ 1.00000000e+00  1.18704517e+00  7.63675324e+00]
 [-3.22462874e-16  4.37016024e-04  1.41421356e-02]]

[[ 1.00000000e+00 -1.00000000e+00  1.35764502e+01]
 [ 1.00000000e+00  1.00000000e+00  7.63675324e+00]
 [ 1.61566460e-15  2.72620899e-16  1.41421356e-02]]

[[ 1.00000000e+00 -1.37059190e+00  1.35764502e+01]
 [ 1.00000000e+00  7.15067863e-01  7.63675324e+00]
 [-8.59059800e-16 -4.37016024e-04  1.41421356e-02]]



## Ex. 4.6

We need to make the function `estimate_b`, which takes a list of homographies `Hs` and returns the vector $\pmb{b}$.

### Response

In [13]:
def get_v(Hi : np.ndarray, alpha : int, beta : int) -> np.ndarray:
    """
        Takes:
            A numpy array (homography): Hi
            
            An integer in range [0, 1]: alpha
            
            An integer in range [0, 1]: beta
        
        Returns:
            A vector (as per week 4, slide 19) of dimensions (1 x 6): v
            
        *Code provided so generously by Jonas Søeborg*
    """
    
    v = np.array(
            [Hi[0, alpha] * Hi[0, beta], Hi[0, alpha] * Hi[1, beta] + Hi[1, alpha] * Hi[0, beta],
            Hi[1, alpha] * Hi[1, beta], Hi[2, alpha] * Hi[0, beta] + Hi[0, alpha] * Hi[2, beta],
            Hi[2, alpha] * Hi[1, beta] + Hi[1, alpha] * Hi[2, beta], Hi[2, alpha] * Hi[2, beta]]
        ) 
    
    return v


def estimate_b(Hs : list) -> np.ndarray:
    """
        Takes:
            A list of homographies: Hs
        
        Returns:
            A vector (as per week 4, slide 22): b
    """
    
    V = []
    for Hi in Hs:
        # vi is a (2 x 6) matrix
        vi = np.array([get_v(Hi, 0, 1), get_v(Hi, 0, 0) - get_v(Hi, 1, 1)]) # <-- Exactly the same as bottom of week 4, slide 21
        V.append(vi)    # <-- appending so we can concatenate all the elements along axis zero later
    
    V = np.vstack(V)    # <-- concatenating along axis zero
    
    # Armed with V, we can now find b using SVD:
    _, _, vh = np.linalg.svd(V)
    b = vh[-1, :].reshape(-1, 1)
    
    return b
    
    

#### Testing whether stuff works

We test by creating a: $\pmb{b}_{true}$ from the $\pmb{K}$ as defined earlier, by utilizing week 4, slide 18:

In [14]:
K_inv = np.linalg.inv(K)
B_true = K_inv.T @ K_inv
b_true = np.array([B_true[0, 0], B_true[0, 1], B_true[1, 1], B_true[0, 2], B_true[1, 2], B_true[2, 2]]).reshape(-1, 1)

b_est = estimate_b(Hs)
b_est_scaled = b_est * (b_true[0, 0] / b_est[0, 0])

print(f"b_true: \n{b_true} \n\nb_est: \n{b_est} \n\nb_est_scaled: \n{b_est_scaled}")

b_true: 
[[ 1.0000e-06]
 [ 0.0000e+00]
 [ 1.0000e-06]
 [-9.6000e-04]
 [-5.4000e-04]
 [ 2.2132e+00]] 

b_est: 
[[-4.51834392e-07]
 [-5.72064877e-19]
 [-4.51834392e-07]
 [ 4.33761016e-04]
 [ 2.43990572e-04]
 [-9.99999876e-01]] 

b_est_scaled: 
[[ 1.00000000e-06]
 [ 1.26609414e-18]
 [ 1.00000000e-06]
 [-9.60000000e-04]
 [-5.40000000e-04]
 [ 2.21320000e+00]]


## Ex. 4.7
Now we define a function called `estimateIntrinsics` which takes a list of homographies `Hs` and returns a camera matrix `K`.


### Response

We use an $\pmb{A}$ matrix defined from Zhang's paper:
$$
    \pmb{A} =

    \begin{bmatrix}
        \alpha  & \gamma    & u_0 \\
        0       & \beta     & v_0 \\
        0       & 0         & 1
    \end{bmatrix}
$$
Where:
$$
    \begin{align*}
        v_0 &= (B_{1, 2} \cdot B_{1, 3} - B_{1, 1} \cdot B_{2, 3}) / (B_{1, 1} \cdot B_{2, 2} - B_{1, 2}^2)
        \\
        \lambda &= B_{3, 3} - \Big(B_{1, 3}^2 + v_0 (B_{1, 2} \cdot B_{1, 3} - B_{1, 1} \cdot B_{2, 3}) \Big) / B_{1, 1}
        \\
        \alpha &= \sqrt{\lambda / B_{1, 1}}
        \\
        \beta &= \sqrt{\lambda \cdot B_{1, 1} / (B_{1, 1} \cdot B_{2, 2} - B_{1, 2}^2)}
        \\
        \gamma &= -B_{1, 2} \cdot \alpha^2 \cdot \beta / \lambda
        \\
        u_0 &= \gamma \cdot v_0 / \beta - B_{1, 3} \cdot \alpha^2 / \lambda
    \end{align*}
$$

In [15]:
def estimateIntrinsics(Hs : list) -> np.ndarray:
    """
        Takes:
            A list of homographies: Hs
            
        Returns:
            A camera matrix (as per Zhangs paper): K
    """
    
    b = estimate_b(Hs)
    B11, B12, B22, B13, B23, B33 = b.flatten()
    
    v0 = (B12 * B13 - B11 * B23) / (B11 * B22 - B12**2)
    lam = B33 - (B13**2 + v0 * (B12 * B13 - B11 * B23)) / B11
    alpha = np.sqrt(lam / B11)
    beta = np.sqrt(lam * B11 / (B11 * B22 - B12**2))
    gamma = -B12 * alpha**2 * beta / lam
    u0 = lam * v0 / beta - B13 * alpha**2 / lam
    
    K = np.array(
        [[alpha,    gamma,  u0],
         [0,        beta,   v0],
         [0,        0,      1]]
    )
    
    return K

In [16]:
K_est = estimateIntrinsics(Hs)
K_est_scaled = K_est * (K[0, 0] / K_est[0, 0])

print(f"True K: \n{K} \n\nEstimated K: \n{K_est} \n\nK_est_scaled: \n{K_est_scaled}")

True K: 
[[1000.    0.  960.]
 [   0. 1000.  540.]
 [   0.    0.    1.]] 

Estimated K: 
[[ 1.00000000e+03 -1.26609414e-09  9.59756009e+02]
 [ 0.00000000e+00  1.00000000e+03  5.40000000e+02]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]] 

K_est_scaled: 
[[ 1.00000000e+03 -1.26609414e-09  9.59756009e+02]
 [ 0.00000000e+00  1.00000000e+03  5.40000000e+02]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]


## Ex. 4.8

Finally we define the function: `Rs, ts = estimateExtrinsics(K, Hs)` that takes camera matrix $\pmb{K}$ and the homographies $\pmb{Hs}$ and returns the rotations $\pmb{Rs}$ and the translations $\pmb{ts}$ of each checkerboard.

### Response
From page six of Zhang's paper we have the following:
$$
    \begin{align*}
        \pmb{r}_1 &= \lambda \pmb{A}^{-1} \pmb{h}_1
        \\
        \pmb{r}_2 &= \lambda \pmb{A}^{-1} \pmb{h}_2
        \\
        \pmb{r}_3 &= \pmb{r}_1 \times \pmb{r}_2
        \\
        \pmb{t} &= \lambda \pmb{A}^{-1} \pmb{h}_3,
    \end{align*}
$$
where
$$
    \begin{gather*}
        \lambda = 1 / ||\pmb{A}^{-1} \pmb{h}_1|| = 1 / || \pmb{A}^{-1} \pmb{h}_2 ||.
    \end{gather*}
$$

REMEBER! The paper refer to our $\pmb{K}$ as $\pmb{A}$.

In [17]:
def estimateExtrinsics(K : np.ndarray, Hs : list) -> tuple[list[np.ndarray], list[np.ndarray]]:
    """
        Takes:
            A camera matrix: K
            
            A list of homographies: Hs
        
        Returns:
            The rotation matrices: Rs
            
            The translations: ts
    """
    
    K_inv = np.linalg.inv(K)
    
    Rs = []
    ts = []
    for Hi in Hs:
        lam = 1/np.linalg.norm(K_inv @ Hi[:, 0])
        r1 = lam * K_inv @ Hi[:, 0]
        r2 = lam * K_inv @ Hi[:, 1]
        r3 = crossOp(r1) @ r2
        t = lam * K_inv @ Hi[:, 2]
        
        if t[2] < 0:
            # This is exactly the same but we should try and estimate again using the negative homography.
            # see week 4, slide 28.
            Hi = -Hi
            lam = 1/np.linalg.norm(K_inv @ Hi[:, 0])
            r1 = lam * K_inv @ Hi[:, 0]
            r2 = lam * K_inv @ Hi[:, 1]
            r3 = crossOp(r1) @ r2
            t = lam * K_inv @ Hi[:, 2]
            
        Rs.append(np.column_stack([r1, r2, r3]))    # <-- column_stack( ) makes the list a matrix where each entry is a column.
        ts.append(t)
         
    
    return Rs, ts

Finally we make a function: `K, Rs, ts = calibratecamera(qs, Q)`, which is the fruit of all our labor.

(Basically it just calls all the functions we already made...)

In [18]:
def calibrateCamera(qs : list, Q : np.ndarray) -> tuple[np.ndarray, list[np.ndarray], list[np.ndarray]]:
    """
        Takes:
            A list of qs for which we find their homographies: qs
            
            A Q matrix: Q
        
        Returns:
            A camera matrix: K
            
            The rotation matrices: Rs
            
            The translations: ts
    """
    
    Hs = estimateHomographies(Q, qs)
    
    K = estimateIntrinsics(Hs)
    
    Rs, ts = estimateExtrinsics(K, Hs)
    
    
    return K, Rs, ts

#### Testing of rotation matrices

In [19]:
R1 = fancyRotate(np.pi/ 10, 0, 0)
R2 = fancyRotate(0, 0, 0)
R3 = fancyRotate(- np.pi/ 10, 0, 0)

Rs_true = [R1, R2, R3]

Rs_est, ts_est = estimateExtrinsics(K, Hs)

for i in range(len(Rs_true)):
    R_est_scaled = Rs_est[i] * (Rs_true[i][0, 0] / Rs_est[i][0, 0])
    print(f"R{i}_true: \n{Rs_true[i]} \n\nR{i}_est: \n{Rs_est[i]} \n\nR{i}_est_scaled: \n{R_est_scaled} \n\n")


R0_true: 
[[ 1.          0.          0.        ]
 [ 0.          0.95105652 -0.30901699]
 [ 0.          0.30901699  0.95105652]] 

R0_est: 
[[ 7.07106781e-01 -6.72498512e-01  2.18508012e-01]
 [ 7.07106781e-01  6.72498512e-01 -2.18508012e-01]
 [-2.28015685e-13  3.09016994e-01  9.51056516e-01]] 

R0_est_scaled: 
[[ 1.00000000e+00 -9.51056516e-01  3.09016994e-01]
 [ 1.00000000e+00  9.51056516e-01 -3.09016994e-01]
 [-3.22462874e-13  4.37016024e-01  1.34499702e+00]] 


R1_true: 
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]] 

R1_est: 
[[ 7.07106781e-01 -7.07106781e-01 -6.71521851e-13]
 [ 7.07106781e-01  7.07106781e-01 -9.44142750e-13]
 [ 1.14244739e-12  1.92772086e-13  1.00000000e+00]] 

R1_est_scaled: 
[[ 1.00000000e+00 -1.00000000e+00 -9.49675309e-13]
 [ 1.00000000e+00  1.00000000e+00 -1.33521948e-12]
 [ 1.61566460e-12  2.72620899e-13  1.41421356e+00]] 


R2_true: 
[[ 1.         -0.          0.        ]
 [ 0.          0.95105652  0.30901699]
 [-0.         -0.30901699  0.95105652]] 

R2_est: 
[[ 7.0

Soooo.... it doesn't really seem like it gives the right results, but apparently we do not care.. for now.